**Necessary installs and imports:**

In [ ]:
# install API libraries
%pip install pillow
%pip install autocorrect
%pip install azure-cognitiveservices-vision-computervision

from autocorrect import Speller
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from msrest.authentication import CognitiveServicesCredentials

from array import array
import os
from PIL import Image
import random
import re
import sys
import shutil
import time

# from google.colab import drive
# drive.mount('/content/drive/')
# os.chdir('/content/drive/MyDrive/')

In [3]:
# API key and endpoint
subscription_key = "redacted from github for security reasons"
endpoint = "https://autoanschecker.cognitiveservices.azure.com/"

# Authenticate the client
computervision_client = ComputerVisionClient(endpoint, CognitiveServicesCredentials(subscription_key))

In [4]:
# prompt to upload image(s) to perform OCR on
from google.colab import files
uploaded = files.upload()

Saving handwritten3.jpg to handwritten3.jpg
Saving ocrpicss2.jpg to ocrpicss2.jpg
Saving Sample-handwritten-text-input-for-OCR.png to Sample-handwritten-text-input-for-OCR.png
Saving WhatsApp Image 2021-02-21 at 10.00.51 PM.jpeg to WhatsApp Image 2021-02-21 at 10.00.51 PM.jpeg
Saving WhatsApp Image 2021-02-21 at 10.00.52 PM.jpeg to WhatsApp Image 2021-02-21 at 10.00.52 PM.jpeg


**Main code**

In [5]:
check = Speller(lang='en')

if uploaded:
    for i, image in enumerate(uploaded, 1):
        # Open the image
        local_image_handwritten = open(image, "rb")

        # Call API with image and raw response (allows you to get the operation location)
        recognize_handwriting_results = computervision_client.read_in_stream(local_image_handwritten, raw=True)

        # Get the operation location (URL with ID as last appendage)
        operation_location_local = recognize_handwriting_results.headers["Operation-Location"]
        # Take the ID off and use to get results
        operation_id_local = operation_location_local.split("/")[-1]

        # Call the "GET" API and wait for the retrieval of the results
        while True:
            recognize_handwriting_result = computervision_client.get_read_result(operation_id_local)
            if recognize_handwriting_result.status not in ['notStarted', 'running']:
                break
            time.sleep(1)

        print(f"===== Extracted text from image #{i}: =====")
        # Print results, line by line
        if recognize_handwriting_result.status == OperationStatusCodes.succeeded:
            for text_result in recognize_handwriting_result.analyze_result.read_results:
                for line in text_result.lines:
                    # print(line.text, sep=' ')        # original OCR'ed line
                    # only autocorrect words which aren't abbreviations.
                    corrected = [word if bool(re.search("[A-Z]+", word)) else check(word) for word in line.text.split() ]
                    print(" ".join(corrected))
        print()
else:
    print("Nothing uploaded!")

===== Extracted text from image #1: =====
TODAY is a
good day!
MY TEAM is
THE bEst TEAM!

===== Extracted text from image #2: =====
Optical Character Recognition
is designed to comment your
handwriting into text.

===== Extracted text from image #3: =====
This is a handwritten
example
Write as good as you can.

===== Extracted text from image #4: =====
A) 1 To perform privileged instruction H/W
provides trap & suetugin from trap
functionality
11) For Context switching of process, H/W
provides timer interrupt
B) 1 When translating visual address to physical
address, it helps ( H/w] in finding whether valid
access has been made on not Tie, the problem
of Segmentation fault]
11 It helps in finding PFE with the help of
VPN and PTE
C ) 1 For both CPU and Memory virtualization
, it provides the type of edit on operation
the program can do [using bits, ie, grfx,
etc??
11) H/W creates User stack & Kernel stack
for both CPU virtualization and
Memogly visualization

===== Extracted text from ima